In [1]:
import torch
import paddle


/usr/lib/python3/dist-packages/urllib3/util/selectors.py:14: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import namedtuple, Mapping
/usr/lib/python3/dist-packages/urllib3/_collections.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping, MutableMapping


In [2]:
def compare(a, b):
    a = torch.from_numpy(a.cpu().numpy()).float()
    b = torch.from_numpy(b.cpu().numpy()).float()
    r1 = str((a - b).abs().mean())
    r2 = str((a - b).abs().max())
    print(r1)
    print(r2)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
torch_weight,torch_bias = torch.load("pt_x.pt")
paddle_weight,paddle_bias = paddle.load("pd_x.pd")
torch_inputs = torch.load("weights.pt")
paddle_inputs = paddle.load("weights.pd")

In [4]:
compare(torch_weight,paddle_weight.t())

tensor(0.)
tensor(0.)


In [5]:
compare(torch_bias,paddle_bias)

tensor(0.)
tensor(0.)


In [6]:
compare(torch_inputs,paddle_inputs)

tensor(0.)
tensor(0.)


In [7]:
shuchu1 = torch.matmul(torch_inputs,torch_weight.cuda().t()) + torch_bias.cuda()
shuchu2 = paddle.matmul(paddle_inputs,paddle_weight) + paddle_bias
compare(shuchu1,shuchu2)

tensor(4.2843e-06)
tensor(3.6240e-05)


In [8]:
# 误差存在,达到10-8级别，还算正常。
import paddle
import torch
import numpy as np

# 正态分布0,0.02
pd_linear_weight = paddle.normal(0,0.02,(768,768))
pd_linear_bias = paddle.normal(0,0.02,(768,))

# 我也定义了类似torch.nn.Linear并使用paddle权重进行初始化
pt_linear_weight = torch.tensor(pd_linear_weight.t().numpy()).cuda()
pt_linear_bias = torch.tensor(pd_linear_bias.numpy()).cuda()

# 初始化输入
# 正态分布0,0.02
pd_inputs = paddle.normal(0,0.02,(32,768)) 
pt_inputs = torch.tensor(pd_inputs.numpy()).cuda()

o1 = paddle.matmul(pd_inputs,pd_linear_weight,transpose_y=True) + pd_linear_bias
o2 = torch.matmul(pt_inputs,pt_linear_weight)  + pt_linear_bias
o1 = o1.numpy()
o2 = o2.cpu().numpy()

print("amax ",np.abs(o1-o2).max())
print("amean ",np.abs(o1-o2).mean())
print(np.all(np.isclose(o1,o2 , atol=0, rtol=1.e-6)))

amax  2.2351742e-08
amean  3.6701877e-09
False


In [9]:
# 误差存在,达到10-5级别，误差太大了。
import paddle
import torch
import numpy as np

# 正态分布0,0.02
pd_linear_weight = paddle.normal(0,1,(768,768))
pd_linear_bias = paddle.normal(0,1,(768,))

# 我也定义了类似torch.nn.Linear并使用paddle权重进行初始化
pt_linear_weight = torch.tensor(pd_linear_weight.t().numpy()).cuda()
pt_linear_bias = torch.tensor(pd_linear_bias.numpy()).cuda()

# 初始化输入
# 正态分布0,0.02
pd_inputs = paddle.normal(0,1,(32,768)) 
pt_inputs = torch.tensor(pd_inputs.numpy()).cuda()

o1 = paddle.matmul(pd_inputs,pd_linear_weight,transpose_y=True) + pd_linear_bias
o2 = torch.matmul(pt_inputs,pt_linear_weight)  + pt_linear_bias
o1 = o1.numpy()
o2 = o2.cpu().numpy()

print("amax ",np.abs(o1-o2).max())
print("amean ",np.abs(o1-o2).mean())
print(np.all(np.isclose(o1,o2 , atol=0, rtol=1.e-6)))

amax  5.722046e-05
amean  9.370713e-06
False


In [11]:
# 误差存在,达到10-4级别，误差太大了。（方差越大，对齐的时候误差也就越大。）
import paddle
import torch
import numpy as np

# 正态分布0,2
pd_linear_weight = paddle.normal(0,2,(768,768))
pd_linear_bias = paddle.normal(0,2,(768,))

# 我也定义了类似torch.nn.Linear并使用paddle权重进行初始化
pt_linear_weight = torch.tensor(pd_linear_weight.t().numpy()).cuda()
pt_linear_bias = torch.tensor(pd_linear_bias.numpy()).cuda()

# 初始化输入
# 正态分布0,2
pd_inputs = paddle.normal(0,2,(32,768)) 
pt_inputs = torch.tensor(pd_inputs.numpy()).cuda()

o1 = paddle.matmul(pd_inputs,pd_linear_weight,transpose_y=True) + pd_linear_bias
o2 = torch.matmul(pt_inputs,pt_linear_weight)  + pt_linear_bias
o1 = o1.numpy()
o2 = o2.cpu().numpy()

print("amax ",np.abs(o1-o2).max())
print("amean ",np.abs(o1-o2).mean())
print(np.all(np.isclose(o1,o2 , atol=0, rtol=1.e-6)))

amax  0.00022888184
amean  3.724343e-05
False
